In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data = pd.read_csv("data/training_data_utf8.csv")

/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head()

,id,price_per_m2,transaction_ym,plot_area,plot_share,property_type,cadastral,district,contract_type,land_use,land_use_share,protection_zone,building_ban,building_height,building_floors,building_year,longitude,latitude
0,7671,224.351484,2013-12,4279,1.0,Betriebsobjekt,Kagran,Donaustadt,Kaufvertrag,GB,1.00,No,No,NaN,1.0,1990.0,16.464163,48.259071
1,7672,357.995227,2013-12,838,1.0,Abbruchobjekt,Liesing,Liesing,Kaufvertrag,W,1.00,No,No,10.5,2.0,NaN,16.275103,48.133264
2,7673,636.474908,2013-12,817,1.0,"Ein-, Zweifamilienhaus",Atzgersdorf,Liesing,Kaufvertrag,GB,1.00,No,No,3.5,2.0,NaN,16.296926,48.145348
3,7674,906.095552,2013-12,607,1.0,"Ein-, Zweifamilienhaus",Ottakring,Ottakring,Kaufvertrag,W,1.00,No,No,7.5,2.0,NaN,16.290968,48.217580
4,7675,167.941498,2013-12,11897,1.0,unbebaut,Groß Jedlersdorf I,Floridsdorf,Kaufvertrag,W,0.19,No,No,NaN,NaN,NaN,16.418471,48.288753


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44366 entries, 0 to 44365
Data columns (total 18 columns):
id                 44366 non-null int64
price_per_m2       44366 non-null float64
transaction_ym     44366 non-null object
plot_area          44366 non-null int64
plot_share         44366 non-null float64
property_type      44366 non-null object
cadastral          44366 non-null object
district           44294 non-null object
contract_type      44366 non-null object
land_use           44260 non-null object
land_use_share     44320 non-null float64
protection_zone    44366 non-null object
building_ban       44366 non-null object
building_height    7800 non-null object
building_floors    18422 non-null float64
building_year      14739 non-null float64
longitude          32465 non-null float64
latitude           32465 non-null float64
dtypes: float64(7), int64(2), object(9)
memory usage: 6.1+ MB


In [5]:
#describe numerical values
data.describe()

,id,price_per_m2,plot_area,plot_share,land_use_share,building_floors,building_year,longitude,latitude
count,44366.000000,44366.000000,4.436600e+04,44366.000000,44320.000000,18422.000000,14739.000000,32465.000000,32465.000000
mean,29853.500000,789.062369,1.880226e+03,0.959631,0.956965,2.063402,1913.283737,16.363182,48.207192
std,12807.505358,2276.246856,1.064498e+04,0.163726,0.144435,1.902943,37.464120,0.074558,0.039031
min,7671.000000,0.161937,1.000000e+00,0.000260,0.000000,0.000000,1800.000000,15.605753,47.850748
25%,18762.250000,181.682084,4.120000e+02,1.000000,1.000000,0.000000,1894.000000,16.310912,48.179708
50%,29853.500000,354.343664,5.810000e+02,1.000000,1.000000,2.000000,1900.000000,16.352831,48.208886
75%,40944.750000,726.728429,1.035000e+03,1.000000,1.000000,4.000000,1940.000000,16.407386,48.230576
max,52036.000000,197628.458498,1.187781e+06,1.000000,1.000000,12.000000,2017.000000,16.572405,48.314311


In [6]:
#describe categorical values
data.describe(include="O")

,transaction_ym,property_type,cadastral,district,contract_type,land_use,protection_zone,building_ban,building_height
count,44366,44366,44366,44294,44366,44260,44366,44366,7800
unique,324,16,88,23,11,56,2,2,97
top,1988-12,unbebaut,Eßling,Donaustadt,Kaufvertrag,WI/GBI,No,No,6.5
freq,296,15345,2270,8476,38853,12099,43306,43857,2763


In [7]:
corrmat = data.corr()

In [8]:
corrmat["price_per_m2"].sort_values(ascending=False)

price_per_m2       1.000000
building_floors    0.385326
plot_share         0.002082
latitude          -0.011045
plot_area         -0.028618
longitude         -0.038409
land_use_share    -0.050230
building_year     -0.051976
id                -0.133134
Name: price_per_m2, dtype: float64

In [9]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

In [10]:
latitude_center = 48.210033
longitude_center = 16.363449

In [11]:
for index, row in data.iterrows():
    data.loc[index, 'distance_to_center'] = haversine(row['longitude'], row['latitude'], longitude_center, latitude_center)

In [12]:
corrmat = data.corr()
corrmat["price_per_m2"].sort_values(ascending=False)

price_per_m2          1.000000
building_floors       0.385326
plot_share            0.002082
latitude             -0.011045
plot_area            -0.028618
longitude            -0.038409
land_use_share       -0.050230
building_year        -0.051976
id                   -0.133134
distance_to_center   -0.268828
Name: price_per_m2, dtype: float64

In [13]:
date = data.transaction_ym.apply(pd.to_datetime, format="%Y-%m")

data["year"] = date.dt.year

corrmat = data.corr()
corrmat["price_per_m2"].sort_values(ascending=False)

price_per_m2          1.000000
building_floors       0.385326
year                  0.131843
plot_share            0.002082
latitude             -0.011045
plot_area            -0.028618
longitude            -0.038409
land_use_share       -0.050230
building_year        -0.051976
id                   -0.133134
distance_to_center   -0.268828
Name: price_per_m2, dtype: float64

In [14]:
relative_share = data.plot_area * data.plot_share
data["relative_share"] = relative_share

corrmat = data.corr()
corrmat["price_per_m2"].sort_values(ascending=False)

price_per_m2          1.000000
building_floors       0.385326
year                  0.131843
plot_share            0.002082
latitude             -0.011045
relative_share       -0.026335
plot_area            -0.028618
longitude            -0.038409
land_use_share       -0.050230
building_year        -0.051976
id                   -0.133134
distance_to_center   -0.268828
Name: price_per_m2, dtype: float64

In [15]:
data.describe()

,id,price_per_m2,plot_area,plot_share,land_use_share,building_floors,building_year,longitude,latitude,distance_to_center,year,relative_share
count,44366.000000,44366.000000,4.436600e+04,44366.000000,44320.000000,18422.000000,14739.000000,32465.000000,32465.000000,32465.000000,44366.000000,4.436600e+04
mean,29853.500000,789.062369,1.880226e+03,0.959631,0.956965,2.063402,1913.283737,16.363182,48.207192,6.212574,1999.322544,1.721025e+03
std,12807.505358,2276.246856,1.064498e+04,0.163726,0.144435,1.902943,37.464120,0.074558,0.039031,3.286353,7.920441,1.027912e+04
min,7671.000000,0.161937,1.000000e+00,0.000260,0.000000,0.000000,1800.000000,15.605753,47.850748,0.042117,1987.000000,7.601351e-02
25%,18762.250000,181.682084,4.120000e+02,1.000000,1.000000,0.000000,1894.000000,16.310912,48.179708,3.230216,1992.000000,3.840000e+02
50%,29853.500000,354.343664,5.810000e+02,1.000000,1.000000,2.000000,1900.000000,16.352831,48.208886,6.312582,1999.000000,5.540000e+02
75%,40944.750000,726.728429,1.035000e+03,1.000000,1.000000,4.000000,1940.000000,16.407386,48.230576,8.686688,2006.000000,9.770000e+02
max,52036.000000,197628.458498,1.187781e+06,1.000000,1.000000,12.000000,2017.000000,16.572405,48.314311,57.364157,2013.000000,1.187781e+06


In [16]:
data = data[data.distance_to_center < 15]
data = data[data.price_per_m2 < 2500]
data.shape

(30441, 21)

In [17]:
data[['building_floors']] = data[['building_floors']].fillna(data[['building_floors']].mean())

In [18]:
corrmat["price_per_m2"].sort_values(ascending=False)

price_per_m2          1.000000
building_floors       0.385326
year                  0.131843
plot_share            0.002082
latitude             -0.011045
relative_share       -0.026335
plot_area            -0.028618
longitude            -0.038409
land_use_share       -0.050230
building_year        -0.051976
id                   -0.133134
distance_to_center   -0.268828
Name: price_per_m2, dtype: float64

In [19]:
data_fillna = data.fillna(data.mean())

In [20]:
corrmat_fillna = data_fillna.corr()
corrmat_fillna["price_per_m2"].sort_values(ascending=False)

price_per_m2          1.000000
building_floors       0.391957
year                  0.246964
plot_share            0.007278
latitude             -0.019270
relative_share       -0.080414
plot_area            -0.086744
building_year        -0.101943
land_use_share       -0.140467
longitude            -0.166063
id                   -0.252815
distance_to_center   -0.487141
Name: price_per_m2, dtype: float64

In [21]:
data_fillna.describe()

,id,price_per_m2,plot_area,plot_share,land_use_share,building_floors,building_year,longitude,latitude,distance_to_center,year,relative_share
count,30441.000000,30441.000000,30441.000000,30441.000000,30441.000000,30441.000000,30441.000000,30441.000000,30441.000000,30441.000000,30441.000000,30441.000000
mean,30511.199369,554.004054,1116.820374,0.962616,0.959225,1.965090,1912.012724,16.363544,48.207234,6.468722,1998.919188,1036.814741
std,12744.549072,485.690583,3441.093298,0.154384,0.141401,1.088691,22.017873,0.076534,0.039983,3.180294,7.867244,3191.748118
min,7671.000000,0.383400,1.000000,0.001000,0.000000,0.000000,1800.000000,16.190731,48.121595,0.042117,1987.000000,1.000000
25%,19680.000000,218.018581,418.000000,1.000000,1.000000,1.965090,1912.012724,16.308571,48.177677,3.627851,1992.000000,392.000000
50%,30717.000000,396.615385,550.000000,1.000000,1.000000,1.965090,1912.012724,16.351679,48.209150,6.631485,1998.000000,532.000000
75%,41657.000000,724.299065,829.000000,1.000000,1.000000,2.000000,1912.012724,16.412278,48.232164,8.808093,2006.000000,794.000000
max,52034.000000,2495.769882,143858.000000,1.000000,1.000000,12.000000,2017.000000,16.546959,48.314311,14.409620,2013.000000,143858.000000


In [22]:
corrmat_fillna = data_fillna.corr()
corrmat_fillna["price_per_m2"].sort_values(ascending=False)

price_per_m2          1.000000
building_floors       0.391957
year                  0.246964
plot_share            0.007278
latitude             -0.019270
relative_share       -0.080414
plot_area            -0.086744
building_year        -0.101943
land_use_share       -0.140467
longitude            -0.166063
id                   -0.252815
distance_to_center   -0.487141
Name: price_per_m2, dtype: float64

In [23]:
train_data = data_fillna[["price_per_m2", "building_floors", "year", "distance_to_center", "land_use", "relative_share", "district"]]
train_data_cat = train_data[["land_use", "district"]]
train_data_num = train_data[["year", "distance_to_center", "building_floors", "relative_share"]]

In [24]:
data = train_data.drop("price_per_m2", axis=1)
labels = train_data["price_per_m2"].copy()

In [25]:
#data["contract_type"] = data["contract_type"].astype(str)
data["land_use"] = data["land_use"].astype(str)
data["district"] = data["district"].astype(str)
#data["protection_zone"] = data["protection_zone"].astype(str)

In [26]:
#This is not important
from future_encoders import OrdinalEncoder

train_data_cat["district"] = data["district"]
train_data_cat["land_use"] = data["land_use"]

encoder = OrdinalEncoder()
housing_cat_encoded = encoder.fit_transform(train_data_cat)
encoder.categories_

/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


[array(['0', '110', 'BG', 'BG/IG', 'E', 'E div.', 'EKL', 'EKLW', 'EPK',
        'ESP', 'EW', 'Ekl', 'Eklw', 'Epk', 'Epl', 'Esp', 'Friedhof', 'G',
        'GB', 'GBBG', 'GBGV', 'GS', 'IG', 'L', 'P', 'SO', 'SO/sonstiges',
        'SPK', 'SWW', 'SWWL', 'Spk', 'Sww', 'SwwL', 'VB', 'Verkehrsband',
        'Vfl', 'W', 'WI/GBI', 'WII/GBII', 'WIII/GBIII', 'WIV/GBIV',
        'WV/GBV', 'WVI/GBVI', 'Wklw', 'nan', 'w', 'we'], dtype=object),
 array(['Alsergrund', 'Brigittenau', 'Donaustadt', 'Döbling', 'Favoriten',
        'Floridsdorf', 'Fünfhaus', 'Hernals', 'Hietzing', 'Innere Stadt',
        'Josefstadt', 'Landstraße', 'Leopoldstadt', 'Liesing',
        'Margareten', 'Mariahilf', 'Meidling', 'Neubau', 'Ottakring',
        'Penzing', 'Simmering', 'Wieden', 'Währing', 'nan'], dtype=object)]

In [27]:
#This is not important
data_fillna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30441 entries, 0 to 44363
Data columns (total 21 columns):
id                    30441 non-null int64
price_per_m2          30441 non-null float64
transaction_ym        30441 non-null object
plot_area             30441 non-null int64
plot_share            30441 non-null float64
property_type         30441 non-null object
cadastral             30441 non-null object
district              30385 non-null object
contract_type         30441 non-null object
land_use              30375 non-null object
land_use_share        30441 non-null float64
protection_zone       30441 non-null object
building_ban          30441 non-null object
building_height       5858 non-null object
building_floors       30441 non-null float64
building_year         30441 non-null float64
longitude             30441 non-null float64
latitude              30441 non-null float64
distance_to_center    30441 non-null float64
year                  30441 non-null int64
relativ

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

In [29]:
from future_encoders import ColumnTransformer
from future_encoders import OneHotEncoder

num_attribs = list(train_data_num)
cat_attribs = list(train_data_cat)

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs)
])

In [30]:
num_attribs

['year', 'distance_to_center', 'building_floors', 'relative_share']

In [31]:
cat_attribs

['land_use', 'district']

In [32]:
data_prepared = full_pipeline.fit_transform(data)
data_prepared

<30441x75 sparse matrix of type '<class 'numpy.float64'>'
	with 182646 stored elements in Compressed Sparse Row format>

In [33]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation", scores.std())

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

lin_reg = LinearRegression()
lin_reg.fit(data_prepared, labels)

scores = cross_val_score(lin_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
lin_reg_rmse_scores = np.sqrt(-scores)

display_scores(lin_reg_rmse_scores)

Scores: [465.47063684 403.45171919 382.77948056 358.3804026  325.2129627
 353.65287303 350.11906838 375.83766258 326.84243795 334.22526143]
Mean: 367.59725052620274
Standard Deviation 40.39690858258576


In [35]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(data_prepared, labels)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [36]:
scores = cross_val_score(ridge_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
ridge_reg_rmse_scores = np.sqrt(-scores)

display_scores(ridge_reg_rmse_scores)

Scores: [465.72205685 403.47787679 382.25622701 357.72999223 325.05948191
 353.87962713 350.09742108 375.77136075 326.9108148  334.3981194 ]
Mean: 367.5302977934822
Standard Deviation 40.44300177787986


In [37]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(data_prepared, labels)

scores = cross_val_score(tree_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
tree_reg_rmse_scores = np.sqrt(-scores)

display_scores(tree_reg_rmse_scores)

Scores: [537.39542038 521.29872995 494.69793354 479.97153867 448.94413669
 443.4705759  433.41068254 459.60241882 461.04836817 401.66478131]
Mean: 468.1504585959291
Standard Deviation 38.977936384481666


In [38]:
feature_importances = tree_reg.feature_importances_
attributes = num_attribs + cat_attribs + list(encoder.categories_[1]) +  list(encoder.categories_[0])
sorted(zip(feature_importances, attributes), reverse=True)

[(0.40892874650995203, 'distance_to_center'),
 (0.20377486198085856, 'year'),
 (0.15663257161045743, 'relative_share'),
 (0.09167644598799164, 'building_floors'),
 (0.01794114867049925, 'SWWL'),
 (0.007893251788944972, 'P'),
 (0.007363400141893408, 'EW'),
 (0.006363301539550394, 'Epk'),
 (0.00523234127654751, 'VB'),
 (0.004859882212856594, 'Verkehrsband'),
 (0.0047744851930872426, 'IG'),
 (0.0046786595151827575, 'Eklw'),
 (0.004458515024527212, 'Epl'),
 (0.004330344364992563, 'GS'),
 (0.004183183023736457, 'Wklw'),
 (0.003997404747298771, 'Spk'),
 (0.00394007588679753, 'Ekl'),
 (0.003749631336138583, 'Vfl'),
 (0.0035980101486040536, 'SWW'),
 (0.0035380722804310306, 'WIII/GBIII'),
 (0.0035242230015343984, 'SPK'),
 (0.0033628787335817055, 'WVI/GBVI'),
 (0.003336736134710806, 'WII/GBII'),
 (0.003305021609356217, 'WIV/GBIV'),
 (0.0032572895286037784, 'WI/GBI'),
 (0.00314594366328923, 'SO'),
 (0.0031435906329005375, 'W'),
 (0.0031306583871449955, 'Meidling'),
 (0.0028860252603712, 'L'),
 (0

In [39]:
#from sklearn.ensemble import RandomForestRegressor

#forest_reg = RandomForestRegressor()
#forest_reg.fit(data_prepared, labels)

#scores = cross_val_score(forest_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
#forest_rmse_scores = np.sqrt(-scores)
#display_scores(forest_rmse_scores)

In [40]:
#feature_importances = forest_reg.feature_importances_
#attributes = num_attribs + cat_attribs + list(encoder.categories_[1]) +  list(encoder.categories_[0])
#sorted(zip(feature_importances, attributes), reverse=True)

In [41]:
corrmat_fillna = data_fillna.corr()
corrmat_fillna["year"].sort_values(ascending=False)

year                  1.000000
price_per_m2          0.246964
building_year         0.206628
distance_to_center    0.162658
plot_share            0.070137
longitude             0.063504
building_floors       0.061755
latitude              0.047504
relative_share        0.020173
plot_area             0.009057
land_use_share       -0.057949
id                   -0.997872
Name: year, dtype: float64